In [10]:
import ee, geemap, folium, pandas, geopandas as gpd, json, datetime, altair as alt, csv
from IPython.display import Image
import ee.mapclient
import datetime
import os
from os.path import exists as file_exists

import requests

# importing pandas as pd
import pandas as pd

In [2]:
yourUserName = "dechastelj" 
yourPassword = "Droneforce1"

In [3]:
os.environ['http_proxy'] = "http://" + yourUserName + ":" + yourPassword + "@web-prdproxy-usr.dmz:80"
os.environ['https_proxy'] = "http://" + yourUserName + ":" + yourPassword + "@web-prdproxy-usr.dmz:80"

In [6]:
#Don't need this every time, initialize can be enough
ee.Authenticate()

Enter verification code: 4/1AfJohXms3o3spRFEa317jD6gPwHscMJZaxPHFs5L8BCHeHPgn8ebceKedGE

Successfully saved authorization token.


In [7]:
ee.Initialize()

In [33]:
# Input files
storage_shapefile = r"c:\temp\GD_Selection.shp"
storage_id_field = "pfi"

# Cloud cover
CLOUD_FILTER = 40 # Under 40%

# UTM Zone
utmZoneInfo = 'EPSG:28355'#z55S


In [34]:
def test_api():
    dem = ee.Image('USGS/SRTMGL1_003')
    xy = ee.Geometry.Point([86.9250, 27.9881])
    elev = dem.sample(xy, 30).first().get('elevation').getInfo()
    print('Authentication active')

#NDVI calculation:
def areadate(img):
    area = img.gt(0).multiply(ee.Image.pixelArea()).divide(1000000).reduceRegion(ee.Reducer.sum(), storage_geometry, 30).get('NDWI');
    return img.set('area', area).set('date', img.get('system:time_start'));

def ndwi_func(i):
    ndwi = i.normalizedDifference (['green', 'swir']).rename('NDWI')
    return i.addBands(ee.Image(ndwi))

def ndvi_func(i):
    ndvi = i.normalizedDifference (['red', 'swir']).rename('NDVI')
    return i.addBands(ee.Image(ndvi))

def mndwi_func(i):
    mndwi = i.normalizedDifference (['nir', 'swir']).rename('MNDWI')
    return i.addBands(ee.Image(mndwi))

def addDate(image):
    # parse date stored in 'system:index'
    date = ee.Date(image.get('system:index'))

    # format date, see http:#www.joda.org/joda-time/apidocs/org/joda/time/format/DateTimeFormat.html
    str = date.format('YYYY-mm-dd')

    return image.set({'Date': str})

def mean_func(image):
    mean = (image                        
            #.mean()
            .reduceRegion(ee.Reducer.mean()))#, storage_geometry, 30))
    mean = mean.getInfo()            
    mean = mean.get('nd')    
    return mean

def year_func(y):
    range = ee.Filter.calendarRange (y, y, 'year')
    mean = (l57ndwi
            .filter(range)
            .select('NDWI')
            .mean()
            .reduceRegion(ee.Reducer.mean(), storage_geometry, 30))
    return mean.set('year', y)

def year_indices(y):
    range = ee.Filter.calendarRange (y, y, 'year')
    ndwimean = (images
        .select('NDWI')
        .filter(range)
        .mean()
        .reduceRegion(ee.Reducer.mean(), storage_geometry, 30))        
    ndwimean = ndwimean.getInfo()

    ndvimean = (images
        .select('NDVI')
        .filter(range)
        .mean()
        .reduceRegion(ee.Reducer.mean(), storage_geometry, 30))        
    ndvimean = ndwimean.getInfo()

    ndwimean = (images
        .select('MNDWI')
        .filter(range)
        .mean()
        .reduceRegion(ee.Reducer.mean(), storage_geometry, 30))        
    mndwimean = ndwimean.getInfo()

    results = [worksNumber,y,ndwimean,ndvimean,mndwimean]
    storageStats.loc[len(storageStats)] = results 
    print(results)
    
def get_area(img):
    total_area = img.multiply(ee.Image.pixelArea())
    ndwi = img.gte(ndwiMaskVal).rename('ndwiMaskLayer')
    ndwi_area = ndwi.multiply(ee.Image.pixelArea())
    return img.set('total_area', total_area).set('ndwi_area', ndwi_area)

def test_api():
    dem = ee.Image('USGS/SRTMGL1_003')
    xy = ee.Geometry.Point([86.9250, 27.9881])
    elev = dem.sample(xy, 30).first().get('elevation').getInfo()
    print('Authentication active')
    
def transformer(feature):
    transformed_feature = feature.transform(utmZoneInfo, 0.001);
    return transformed_feature;

#def get_area(feature):
#    area = feature.area()
#    area = ee.Number(area).divide(1e6).round()
#    return area

def extract_water(img):
    ndwi_image = img.normalizedDifference(['B2', 'B5'])
    water_image = ndwi_image.gt(ndwiMaskVal)
    return water_image

def get_stats(image):
    date = ee.Date(image.get('system:time_start'))        
    date = date.format('dd-MM-YYYY')
    allPixels = image.get('allPixels')
    allPixels = allPixels.getInfo()
    ndwiPixels = image.get('ndwiPixels')
    ndwiPixels = ndwiPixels.getInfo()
    CLOUD_COVER = image.get('CLOUD_COVER')
    CLOUD_COVER = CLOUD_COVER.getInfo()
    return date, allPixels, ndwiPixels, CLOUD_COVER

def rename_bands(bands):
    image.select(bands,STD_NAMES)
    return image
    

def filterImages(storage_geometry):
    # Get band names
    S2_BANDS = ['B2',   'B3',    'B4',  'B8',  'B11']; #Sentinel 2
    LC8_BANDS = ['B2',   'B3',    'B4',  'B5',  'B6']; #Landsat 8
    LC457_BANDS = ['B1',   'B2',    'B3',  'B4',  'B5']; #Landsat 4,5,7
    STD_NAMES = ['blue', 'green', 'red', 'nir', 'swir'] # Standard names
    
    # Load Landsat 5, 7, 8
    s2 = ee.ImageCollection("COPERNICUS/S2_SR").filterBounds(storage_geometry).select(['B2','B3','B4','B8','B11'])
    l4 = ee.ImageCollection("LANDSAT/LT04/C02/T1_L2").filterBounds(storage_geometry).select(['B1','B2','B3','B4','B5'])
    l5 = ee.ImageCollection("LANDSAT/LT05/C02/T1_L2").filterBounds(storage_geometry).select(['B1','B2','B3','B4','B5'])
    l7 = ee.ImageCollection("LANDSAT/LE07/C02/T1_L2").filterBounds(storage_geometry).select(['B1','B2','B3','B4','B5'])
    l8 = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2").filterBounds(storage_geometry).select(['B2','B3','B4','B5','B6'])
    
    # Rename bands
    s2 = s2.map(lambda image: image.select(S2_BANDS, STD_NAMES))
    l4 = l4.map(lambda image: image.select(LC457_BANDS, STD_NAMES))
    l5 = l5.map(lambda image: image.select(LC457_BANDS, STD_NAMES))
    l7 = l7.map(lambda image: image.select(LC457_BANDS, STD_NAMES))
    l8 = l8.map(lambda image: image.select(LC8_BANDS, STD_NAMES))
    
    # Filter images by cloud cover
    s2 = s2.filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', CLOUD_FILTER))
    l8 = l8.filter(ee.Filter.lt('CLOUD_COVER', CLOUD_FILTER))
    l7 =l7.filter(ee.Filter.lt('CLOUD_COVER', CLOUD_FILTER))
    l5 = l5.filter(ee.Filter.lt('CLOUD_COVER', CLOUD_FILTER))
    l4 = l4.filter(ee.Filter.lt('CLOUD_COVER', CLOUD_FILTER))
    
    # Merge collections
    images = l4.merge(l5).merge(l7).merge(l8).merge(s2)
    # Clip images to storage
    images = images.map(lambda image: image.clip(storage_geometry).reproject(crs=utmZoneInfo))
    #.filterDate(start_date, end_date)
    # Add date
    images.map(addDate)
    # Sort by date
    images = images.sort('Date')
    # Add indices
    images.map(ndwi_func).map(ndvi_func).map(mndwi_func)
    return images

def exportStats(worksNumber):
    # Create a new dataframe
    storageStats = pandas.DataFrame(columns=['Works Number', 'Year', 'NDWI', 'NDVI', 'MNDWI'])        
    # Select the feature
    storage_selection = storage_collection.filter(ee.Filter.eq("Wks_Num_tx", worksNumber))   
    # Get the geometry
    storage_geometry = storage_selection.geometry()
    # Get the area of the storage
    storage_area = storage_geometry.area(1).getInfo()
    # Create list of images
    images = filterImages(storage_geometry)           
    # Create a list of images
    imageSize = images.size().getInfo()    
    imageList = images.toList(images.size())
    # Get stats ##
    # For each image
    for i in range(0, imageSize):    
        # Choose the image
        image = ee.Image(imageList.get(i))
        # Get date
        date = ee.Date(image.get('system:time_start')).format("yyyy-MM-dd")
        date = date.getInfo()
        # Create images
        ndvi_image = image.select("NDVI")
        ndwi_image = image.select("NDWI")
        mndwi_image = image.select("MNDWI")
        #ndvi_image = image.normalizedDifference(['B4', 'B3'])
        #ndwi_image = image.normalizedDifference(['B4', 'B5'])
        #mndwi_image = image.normalizedDifference(['B2', 'B5'])
        # Get the area of the storage
        storage_area = storage_geometry.area(1).getInfo()
        # Get water area - Mask water areas using MNDWI
        water_image = mndwi_image.gt(ndwiMaskVal).selfMask()
        water_vector = water_image.reduceToVectors()
        water_area = water_vector.geometry().area(1).getInfo()
        # Get average ndvi, ndwi and mndwi values            
        ndvi = mean_func(ndvi_image)
        ndwi = mean_func(ndwi_image)
        mndwi = mean_func(mndwi_image)
        # Get cloud cover
        cloudiness = image.get("CLOUD_COVER").getInfo()
        # Add results to storage stats dataframe
        results = [worksNumber,date,storage_area,water_area,ndvi,ndwi,mndwi,cloudiness]
        print(results)
        storageStats.loc[len(storageStats)] = results   
        # Export image
        #exportRGBImage(image)
   
    # Save to CSV
    storageStats.to_csv(outputCSV)

def daily_stats(image):
    # Get date
    date = ee.Date(image.get('system:time_start')).format("yyyy-MM-dd")
    date = date.getInfo()
    # Create images
    ndvi_image = image.select('NDVI')
    ndwi_image = image.select('NDWI')
    mndwi_image = image.select('MNDWI')
    # Get water area - Mask water areas using MNDWI
    water_image = mndwi_image.gt(ndwiMaskVal).selfMask()
    water_vector = water_image.reduceToVectors()
    water_area = water_vector.geometry().area(1).getInfo()
    # Get average ndvi, ndwi and mndwi values            
    ndvi = mean_func(ndvi_image)
    ndwi = mean_func(ndwi_image)
    mndwi = mean_func(mndwi_image)
    # Get cloud cover
    try:
        cloudiness = image.get("CLOUD_COVER").getInfo()
    except:
        cloudiness = image.get("CLOUDY_PIXEL_PERCENTAGE").getInfo()
    # Add results to storage stats dataframe
    results = [worksNumber,date,storage_area,water_area,ndvi,ndwi,mndwi,cloudiness]
    return results
    #print(results)
    #storageStats.loc[len(storageStats)] = results   
    # Export image
    #exportRGBImage(image)
    
    
def exportRGBImage(worksNumber):
    # Select the feature
    storage_selection = storage_collection.filter(ee.Filter.eq("Wks_Num_tx", worksNumber))   
    # Get the geometry
    storage_geometry = storage_selection.geometry()
    storage_geometry = storage_geometry.buffer(distance=500)
    storage_geometry = storage_geometry.bounds()
    # Get the latest image
    s2 = ee.ImageCollection("COPERNICUS/S2_SR").filterBounds(storage_geometry).select(['B2','B3','B4'])
    # Filter images by cloud cover
    s2 = s2.filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 1))
    # Get the latest image
    imageSize = s2.size().getInfo()    
    imageList = s2.toList(s2.size())
    # Choose the image
    image = ee.Image(imageList.get(imageSize-1))        
    # Create a RGB image for export
    stats = image.reduceRegion(reducer=ee.Reducer.minMax(),geometry=storage_geometry,scale=100,bestEffort=True)
    statDict = stats.getInfo()
    try: 
        prettyImg = image.visualize(bands=['B4', 'B3', 'B2'],                                 
             min=[statDict['B4_min'],statDict['B3_min'],statDict['B2_min']],
             max=[statDict['B4_max'],statDict['B3_max'],statDict['B2_max']],
             gamma=1)
        
        # Export multi-band GeoTIFF file.
        url = prettyImg.getDownloadUrl({
            #'bands': ['B4', 'B3', 'B2'],
            'region': storage_geometry,
            #'scale': 10,
            'format': 'GEO_TIFF'
        })            
        response = requests.get(url)
        with open(filename, 'wb') as fd:                
          fd.write(response.content)
    except:
        print('Export failed')

    return

def exportMap(worksNumber, start_date, end_date):
    # Select the feature
    storage_selection = storage_collection.filter(ee.Filter.eq(storage_id_field, worksNumber))   
    # Get the geometry
    storage_geometry = storage_selection.geometry()
    storage_geometry = storage_geometry.buffer(distance=500)
    storage_geometry = storage_geometry.bounds()
    
    # Load Landsat 5 & 7
    l5 = ee.ImageCollection("LANDSAT/LT05/C01/T1_SR")
    l7 = ee.ImageCollection("LANDSAT/LE07/C01/T1_SR")

    # Merge Landsat 5 & 7
    l57merge = l5.merge(l7) 
    
    # Filter landsat collections by area, cloud cover and bands required for NDWI
    l57filter = (l57merge.filterBounds(storage_geometry)
              .filterDate(start_date, end_date)
              .map(lambda image: image.clip(storage_geometry).reproject(crs=utmZoneInfo))
              .filterMetadata('CLOUD_COVER', 'less_than', CLOUD_FILTER))   
    
    # Get the latest image
    imageSize = l57filter.size().getInfo()    
    imageList = l57filter.toList(l57filter.size())
    # Choose the image
    image = ee.Image(imageList.get(imageSize-1))        
    # Create a RGB image for export
    stats = image.reduceRegion(reducer=ee.Reducer.minMax(),geometry=storage_geometry,scale=100,bestEffort=True)
    statDict = stats.getInfo()
    try: 
        prettyImg = image.visualize(bands=['B3', 'B2', 'B1'],                                 
             min=[statDict['B3_min'],statDict['B2_min'],statDict['B1_min']],
             max=[statDict['B3_max'],statDict['B2_max'],statDict['B1_max']],
             gamma=1)
        
        # Export multi-band GeoTIFF file.
        url = prettyImg.getDownloadUrl({
            #'bands': ['B4', 'B3', 'B2'],
            'region': storage_geometry,
            #'scale': 10,
            'format': 'GEO_TIFF'
        })            
        response = requests.get(url)
        with open(filename, 'wb') as fd:                
          fd.write(response.content)
    except:
        print('Export failed')

    return

In [18]:
# Load shapefile into geopandas
storage_shapes = gpd.read_file(storage_shapefile)

# Convert shapes into json
json_storages = json.loads(storage_shapes.to_json())

# Convert json into feature collection
storage_collection =  ee.FeatureCollection(json_storages)

# Reproject collection
storage_collection = storage_collection.map(transformer);

# Add area
storage_collection.map(lambda feature: feature.set('total_area', ee.Number(feature.area()).divide(1e6).round()))

In [19]:
# list the unique quality code values
worksSet = sorted(storage_shapes[storage_id_field].unique())
print(worksSet)

['114006000158007', '114006000158372', '114006000158978', '114006000162816', '114006000177675', '114006000177767', '114006000181885', '114006000181892', '114006000190585', '114006000190718', '114006000193888', '114006000240463', '114006000240602', '114006000248747', '114006000252050', '114006000252234', '114006000252466', '114006000252499', '114006000252649', '114006000252990', '114006000253029', '114006000256234', '114006000256287', '114006000256892', '114006000257191', '114006000257283', '114006000258733', '114006000258743', '114006000259360', '114006000300850', '114006000303528', '114006000304299', '114006000304845', '114006000305017', '114006000305172', '114006000305412']


In [ ]:
# Start timer
startNow = datetime.datetime.now()
print("Starting at " + str(startNow))

years = [1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022]

# For each storage
for worksNumber in worksSet:
    
    for year in years:
        
        print(year)
        

        # Dates
        start_date = str(year) + '-01-01'
        end_date = str(year+1) + '-01-01'
        print(start_date)
        print(end_date)

        # Create a image filename
        filename = "output\\" + worksNumber + "_" + year + ".png"

        # Check if already completed
        if file_exists(filename):

            print("Skipping: " + filename)

        else:

            # Start process
            print("Exporting: " + filename)

            # Create an image function to map
            exportMap(worksNumber, start_date, end_date)
    
# Print the storage stats
print("Finished")

# Finish time
endNow = datetime.datetime.now()
diff = (endNow-startNow).total_seconds()
print("End at " + str(endNow))
print("Taking: " + str(diff/60) + " minutes")

Starting at 2023-10-26 11:49:21.492215
1987
1987-01-01
1988-01-01
Exporting: output\114006000158007.png
1988
1988-01-01
1989-01-01
Skipping: output\114006000158007.png
1989
1989-01-01
1990-01-01
Skipping: output\114006000158007.png
1990
1990-01-01
1991-01-01
Skipping: output\114006000158007.png
1991
1991-01-01
1992-01-01
Skipping: output\114006000158007.png
1992
1992-01-01
1993-01-01
Skipping: output\114006000158007.png
1993
1993-01-01
1994-01-01
Skipping: output\114006000158007.png
1994
1994-01-01
1995-01-01
Skipping: output\114006000158007.png
1995
1995-01-01
1996-01-01
Skipping: output\114006000158007.png
1996
1996-01-01
1997-01-01
Skipping: output\114006000158007.png
1997
1997-01-01
1998-01-01
Skipping: output\114006000158007.png
1998
1998-01-01
1999-01-01
Skipping: output\114006000158007.png
1999
1999-01-01
2000-01-01
Skipping: output\114006000158007.png
2000
2000-01-01
2001-01-01
Skipping: output\114006000158007.png
2001
2001-01-01
2002-01-01
Skipping: output\114006000158007.png


1988
1988-01-01
1989-01-01
Skipping: output\114006000177675.png
1989
1989-01-01
1990-01-01
Skipping: output\114006000177675.png
1990
1990-01-01
1991-01-01
Skipping: output\114006000177675.png
1991
1991-01-01
1992-01-01
Skipping: output\114006000177675.png
1992
1992-01-01
1993-01-01
Skipping: output\114006000177675.png
1993
1993-01-01
1994-01-01
Skipping: output\114006000177675.png
1994
1994-01-01
1995-01-01
Skipping: output\114006000177675.png
1995
1995-01-01
1996-01-01
Skipping: output\114006000177675.png
1996
1996-01-01
1997-01-01
Skipping: output\114006000177675.png
1997
1997-01-01
1998-01-01
Skipping: output\114006000177675.png
1998
1998-01-01
1999-01-01
Skipping: output\114006000177675.png
1999
1999-01-01
2000-01-01
Skipping: output\114006000177675.png
2000
2000-01-01
2001-01-01
Skipping: output\114006000177675.png
2001
2001-01-01
2002-01-01
Skipping: output\114006000177675.png
2002
2002-01-01
2003-01-01
Skipping: output\114006000177675.png
2003
2003-01-01
2004-01-01
Skipping: out

1988
1988-01-01
1989-01-01
Skipping: output\114006000190585.png
1989
1989-01-01
1990-01-01
Skipping: output\114006000190585.png
1990
1990-01-01
1991-01-01
Skipping: output\114006000190585.png
1991
1991-01-01
1992-01-01
Skipping: output\114006000190585.png
1992
1992-01-01
1993-01-01
Skipping: output\114006000190585.png
1993
1993-01-01
1994-01-01
Skipping: output\114006000190585.png
1994
1994-01-01
1995-01-01
Skipping: output\114006000190585.png
1995
1995-01-01
1996-01-01
Skipping: output\114006000190585.png
1996
1996-01-01
1997-01-01
Skipping: output\114006000190585.png
1997
1997-01-01
1998-01-01
Skipping: output\114006000190585.png
1998
1998-01-01
1999-01-01
Skipping: output\114006000190585.png
1999
1999-01-01
2000-01-01
Skipping: output\114006000190585.png
2000
2000-01-01
2001-01-01
Skipping: output\114006000190585.png
2001
2001-01-01
2002-01-01
Skipping: output\114006000190585.png
2002
2002-01-01
2003-01-01
Skipping: output\114006000190585.png
2003
2003-01-01
2004-01-01
Skipping: out